# 003 List all the actors who acted in a film before 1970 and in a film after 1990. That is greater than 1970 and less than 1990

In [1]:
spark

org.apache.spark.sql.SparkSession@6b9c9741

In [2]:
import org.apache.spark.sql.functions._

In [3]:
import org.apache.spark.sql.types._

In [4]:
import org.apache.spark.sql.expressions.Window

In [5]:
val spark=SparkSession.builder().appName("movies app").master("local[*]").getOrCreate()

spark = org.apache.spark.sql.SparkSession@6b9c9741


org.apache.spark.sql.SparkSession@6b9c9741

In [6]:
import spark.implicits._

# UDFs

In [7]:
val getYear=udf((year:String)=>{
    val p="\\d{4}".r
    val yr=p.findFirstMatchIn(year).getOrElse("").toString
    if(yr!=""){
        yr.toInt
    }
    else{
        0
    }
})

getYear = UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))

# loading movies

In [8]:
val Movie=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Movie.csv")
.dropDuplicates()

Movie = [index: string, MID: string ... 4 more fields]


[index: string, MID: string ... 4 more fields]

# printing table structure to know what are the columns and it's data type

In [9]:
Movie.printSchema

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- num_votes: string (nullable = true)



# Displaying first two records to see sample data
# observation 1: year,rating,num_votes,index columns data types should be Integer
# observation 2: also we need do trim to remove any extra spaces

In [10]:
Movie.show(2)

+-----+---------+-----------------+----+------+---------+
|index|      MID|            title|year|rating|num_votes|
+-----+---------+-----------------+----+------+---------+
|   85|tt3224288|Beyond the Clouds|2017|   7.0|     1123|
|  913|tt2178508|   Son of Sardaar|2012|   4.0|     7895|
+-----+---------+-----------------+----+------+---------+
only showing top 2 rows



# displaying total records in the table

In [11]:
Movie.count

3475

# as per observation need to change the data type

In [12]:
val movies=Movie.withColumn("year",getYear(Movie("year")).cast(IntegerType))
.withColumn("title",trim(Movie("title")))
.withColumn("MID",trim(Movie("MID")))
.withColumn("rating",trim(Movie("rating")).cast(DoubleType))
.withColumn("num_votes",trim(Movie("num_votes")).cast(LongType))
.withColumn("index",trim(Movie("index")).cast(IntegerType))
.dropDuplicates("MID")

movies = [index: int, MID: string ... 4 more fields]


[index: int, MID: string ... 4 more fields]

# now print the schema to verify columns and it's data type

In [13]:
movies.printSchema

root
 |-- index: integer (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- rating: double (nullable = true)
 |-- num_votes: long (nullable = true)



# print the total number of records in the table after data cleaning

In [14]:
movies.count

3475

# show the first two records as a sample data

In [15]:
movies.show(2)

+-----+---------+--------------------+----+------+---------+
|index|      MID|               title|year|rating|num_votes|
+-----+---------+--------------------+----+------+---------+
| 3008|tt0036077|              Kismet|1943|   7.5|       71|
| 2969|tt0102701|Prahaar: The Fina...|1991|   7.9|     1696|
+-----+---------+--------------------+----+------+---------+
only showing top 2 rows



# Load the Person.csv file and it has header and store theses records in Person variable

In [16]:
val Person=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Person.csv")

Person = [index: string, PID: string ... 2 more fields]


[index: string, PID: string ... 2 more fields]

# Print the schema of Person variable,first two records and it's count

In [17]:
Person.show(2)
Person.printSchema
val p1Count=Person.count

+-----+---------+---------------+------+
|index|      PID|           Name|Gender|
+-----+---------+---------------+------+
|    0|nm0000288| Christian Bale|  Male|
|    1|nm0000949| Cate Blanchett|Female|
+-----+---------+---------------+------+
only showing top 2 rows

root
 |-- index: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p1Count = 38285


38285

# Data Cleaning
# change the data types as per data and also remove extra spaces from data

In [18]:
val persons=Person.withColumn("PID",trim(Person("PID")))
.withColumn("Name",trim(Person("Name")))
.withColumn("Gender",trim(Person("Gender")))
.withColumn("index",trim(Person("index")).cast(IntegerType))
.dropDuplicates("PID")

persons = [index: int, PID: string ... 2 more fields]


[index: int, PID: string ... 2 more fields]

# show the sample data, schema and it's count of variable persons

In [19]:
persons.show(2)
persons.printSchema
val p2Count=persons.count

+-----+---------+-------------+------+
|index|      PID|         Name|Gender|
+-----+---------+-------------+------+
|15793|nm0022343|       Alpana|Female|
| 3121|nm0028416|Martin Andris|  Male|
+-----+---------+-------------+------+
only showing top 2 rows

root
 |-- index: integer (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p2Count = 37566


37566

# print the difference between Person and persons records count

In [20]:
val diff=p1Count-p2Count
println(diff+" duplicates found")

719 duplicates found


diff = 719


719

# Loading and cleaning M_Cast table

In [21]:
val M_Cast=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/M_Cast.csv")

M_Cast = [index: string, MID: string ... 2 more fields]


[index: string, MID: string ... 2 more fields]

# Sample data from M_Cast table

In [22]:
M_Cast.printSchema
M_Cast.show(2)
val m_c1Count=M_Cast.count

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: string (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|    0|tt2388771| nm0000288|  0|
|    1|tt2388771| nm0000949|  1|
+-----+---------+----------+---+
only showing top 2 rows



m_c1Count = 82837


82837

# Cleaning data m_cast

In [23]:
val m_cast=M_Cast.withColumn("ID",trim(col("ID")).cast(LongType))
.withColumn("index",trim(col("index")).cast(LongType))
.withColumn("MID",trim(col("MID")))
.withColumn("PID",trim(col("PID")))
.dropDuplicates("MID","PID")
//.dropDuplicates("MID","PID","ID")
.dropDuplicates("ID")
.dropDuplicates()

m_cast = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

# Sample data after data cleaning for m_cast

In [24]:
m_cast.printSchema
m_cast.show(2)
val m_c2Count=m_cast.count

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|   26|tt2388771|nm10302077| 26|
|   29|tt2388771|nm10302080| 29|
+-----+---------+----------+---+
only showing top 2 rows



m_c2Count = 82837


82837

# Print difference of total count after and before data cleaning

In [25]:
val diff_m_cast_count=m_c1Count-m_c2Count
println(diff_m_cast_count+" d")

0 d


diff_m_cast_count = 0


0

# by default many of us will try to write like below
# it will not work since there is such movie

In [26]:
val MIDs=movies.filter($"year".between(1970 && $"year">1990).select("MID")
MIDs.count
MIDs.show

Name: Compile Error
Message: <console>:2: error: ')' expected but '.' found.
MIDs.count
    ^
<console>:3: error: ';' expected but '.' found.
MIDs.show
    ^

StackTrace: 

# by default many of us will try to write like below
# it will not work since our condition either or. both conditions are mandatory

In [27]:
val MIDs=movies.filter($"year".between(1970 || $"year">1990).select("MID")
MIDs.count
MIDs.show

Name: Compile Error
Message: <console>:2: error: ')' expected but '.' found.
MIDs.count
    ^
<console>:3: error: ';' expected but '.' found.
MIDs.show
    ^

StackTrace: 

# Lets start from here towards the solution

# Join the persons table with m_cast to get the movie id and actors
# now join the result above with movie to get year with the result

In [28]:
val actorsWithMovies=persons.join(m_cast,"PID").dropDuplicates().join(movies,"MID")
.select("MID","PID","year","Name","Gender")

actorsWithMovies = [MID: string, PID: string ... 3 more fields]


[MID: string, PID: string ... 3 more fields]

# look at sample data after joining

In [29]:
actorsWithMovies.orderBy("PID").show(10)

+---------+---------+----+-----------------+------+
|      MID|      PID|year|             Name|Gender|
+---------+---------+----+-----------------+------+
|tt0053126|nm0000002|1959|    Lauren Bacall|Female|
|tt0087892|nm0000027|1984|    Alec Guinness|  Male|
|tt0046427|nm0000039|1953|     Deborah Kerr|Female|
|tt0046427|nm0000042|1953|        Alan Ladd|  Male|
|tt0066070|nm0000047|1970|     Sophia Loren|Female|
|tt0031580|nm0000073|1939|   Shirley Temple|Female|
|tt0075860|nm0000076|1977|François Truffaut|  null|
|tt0327437|nm0000092|2004|      John Cleese|  Male|
|tt0120102|nm0000093|1997|        Brad Pitt|  null|
|tt4977530|nm0000096|2017| Gillian Anderson|Female|
+---------+---------+----+-----------------+------+
only showing top 10 rows



# get only 1970 movie MIDs only

In [30]:
val movies_1970_list=actorsWithMovies.filter($"year"<1970).select("PID","year","Name","gender").dropDuplicates()


movies_1970_list = [PID: string, year: int ... 2 more fields]


[PID: string, year: int ... 2 more fields]

# look at our sample data 

In [31]:
movies_1970_list.sort(desc("year")).show

+---------+----+----------------+------+
|      PID|year|            Name|gender|
+---------+----+----------------+------+
|nm0771700|1969|Jürgen Schilling|  Male|
|nm0474894|1969|    Sudhir Kumar|  Male|
|nm0810994|1969|        Snehlata|Female|
|nm0451296|1969|     Rashid Khan|  Male|
|nm1524864|1969|    Baby Ruksana|  Male|
|nm1542598|1969|     Ajay Kumari|  Male|
|nm1189876|1969|   Bishan Khanna|  Male|
|nm1291858|1969|     Vishal Deep|  Male|
|nm0534867|1969|           Madhu|  Male|
|nm1417125|1969|          Sushma|Female|
|nm0403827|1969|   Nasir Hussain|  Male|
|nm1259556|1969|    Vinod Sharma|  Male|
|nm0695199|1969|            Pran|  Male|
|nm0080445|1969|  Bharat Bhushan|  Male|
|nm1750628|1969|    Master Anwar|  Male|
|nm4754862|1969|       Prema Rao|Female|
|nm1429330|1969|           Qamar|  Male|
|nm0223607|1969|          Dhumal|  Male|
|nm1566214|1969|   Master Ripple|  Male|
|nm1306744|1969|        Meena T.|Female|
+---------+----+----------------+------+
only showing top

# get only 1990 movie MIDs only and modify column names to avoid futur join

In [32]:
val movies_1990_list=actorsWithMovies.filter($"year">1990).select($"PID".alias("PID_90"),$"Name".alias("Name_90"),$"year".alias("year_90"))
.dropDuplicates()


movies_1990_list = [PID_90: string, Name_90: string ... 1 more field]


[PID_90: string, Name_90: string ... 1 more field]

# look at our sample data 

In [33]:
movies_1990_list.sort("year").show

+---------+------------------+-------+
|   PID_90|           Name_90|year_90|
+---------+------------------+-------+
|nm1575779|        Shamsuddin|   1991|
|nm0622186|         Alok Nath|   1991|
|nm0422586|     Ayesha Jhulka|   1991|
|nm1566068|  Satyendra Kapoor|   1991|
|nm0474886|      Sharad Kumar|   1991|
|nm0004469|Srinivasa Rao Kota|   1991|
|nm0665350|       Amrit Patel|   1991|
|nm1146435|  Nagesh S. Vagade|   1991|
|nm1361028|    Kumud Tripathi|   1991|
|nm0665544|       Dina Pathak|   1991|
|nm0540990|            Mamaji|   1991|
|nm1550918|        R.C. Nanda|   1991|
|nm0710037|  Sakshi Ranga Rao|   1991|
|nm1024811|          Manmauji|   1991|
|nm0802107|        Dara Singh|   1991|
|nm0347901|     Rakhee Gulzar|   1991|
|nm1017480|         Abbas Ali|   1991|
|nm0031924|         Aparajita|   1991|
|nm0788906|     Romesh Sharma|   1991|
|nm0659156|   Aditya Pancholi|   1991|
+---------+------------------+-------+
only showing top 20 rows



# now join the 1970-90 movies based on PID column

In [34]:
val movies_1970_90_list=movies_1970_list.join(movies_1990_list,$"PID"===$"PID_90","inner").select("PID","name","year","Gender").dropDuplicates("PID")


movies_1970_90_list = [PID: string, name: string ... 2 more fields]


[PID: string, name: string ... 2 more fields]

# look at our final result 

In [35]:
movies_1970_90_list.sort("PID","year").show

+---------+------------------+----+------+
|      PID|              name|year|Gender|
+---------+------------------+----+------+
|nm0000821|  Amitabh Bachchan|1969|  Male|
|nm0003987|Mohandas K. Gandhi|1965|  null|
|nm0004334|             Rekha|1968|Female|
|nm0004429|        Dharmendra|1962|  Male|
|nm0004432| Prithviraj Kapoor|1964|  Male|
|nm0004433|     Shammi Kapoor|1962|  Male|
|nm0004434|     Shashi Kapoor|1967|  Male|
|nm0004435|     Rajesh Khanna|1967|  Male|
|nm0004564|       Hema Malini|1968|Female|
|nm0004569|       Sanjay Dutt|1964|  Male|
|nm0004570|        Sunil Dutt|1963|  Male|
|nm0006348|    Rajendra Kumar|1960|  Male|
|nm0006369|       Ashok Kumar|1949|  Male|
|nm0006370|       Manoj Kumar|1968|  Male|
|nm0007131|      Subhash Ghai|1969|  null|
|nm0007147|         Dev Anand|1952|  Male|
|nm0007807|             Abbas|1961|  Male|
|nm0008248|             Abdul|1949|  null|
|nm0013026|        Jalal Agha|1969|  Male|
|nm0014973|              Ajay|1955|  Male|
+---------+

# Validation 1

In [36]:
actorsWithMovies.filter($"PID"==="nm0000821").sort("year").show(1000)

+---------+---------+----+----------------+------+
|      MID|      PID|year|            Name|Gender|
+---------+---------+----+----------------+------+
|tt0064915|nm0000821|1969|Amitabh Bachchan|  Male|
|tt0066763|nm0000821|1971|Amitabh Bachchan|  Male|
|tt0067636|nm0000821|1971|Amitabh Bachchan|  Male|
|tt0067164|nm0000821|1971|Amitabh Bachchan|  Male|
|tt0067555|nm0000821|1971|Amitabh Bachchan|  Male|
|tt0067668|nm0000821|1971|Amitabh Bachchan|  Male|
|tt0068257|nm0000821|1972|Amitabh Bachchan|  Male|
|tt0068532|nm0000821|1972|Amitabh Bachchan|  Male|
|tt0158092|nm0000821|1972|Amitabh Bachchan|  Male|
|tt0069218|nm0000821|1972|Amitabh Bachchan|  Male|
|tt0069152|nm0000821|1972|Amitabh Bachchan|  Male|
|tt0068620|nm0000821|1972|Amitabh Bachchan|  Male|
|tt0068305|nm0000821|1972|Amitabh Bachchan|  Male|
|tt0070434|nm0000821|1973|Amitabh Bachchan|  Male|
|tt0070637|nm0000821|1973|Amitabh Bachchan|  Male|
|tt0069751|nm0000821|1973|Amitabh Bachchan|  Male|
|tt0070947|nm0000821|1973|Amita

# Validation 2

In [37]:
actorsWithMovies.filter($"PID"==="nm0004429").sort("year").show

+---------+---------+----+----------+------+
|      MID|      PID|year|      Name|Gender|
+---------+---------+----+----------+------+
|tt0206554|nm0004429|1962|Dharmendra|  Male|
|tt0056850|nm0004429|1963|Dharmendra|  Male|
|tt0154565|nm0004429|1964|Dharmendra|  Male|
|tt0233147|nm0004429|1964|Dharmendra|  Male|
|tt0139398|nm0004429|1965|Dharmendra|  Male|
|tt0229952|nm0004429|1966|Dharmendra|  Male|
|tt0060145|nm0004429|1966|Dharmendra|  Male|
|tt0139525|nm0004429|1966|Dharmendra|  Male|
|tt0137361|nm0004429|1966|Dharmendra|  Male|
|tt0060306|nm0004429|1966|Dharmendra|  Male|
|tt0060659|nm0004429|1966|Dharmendra|  Male|
|tt0139447|nm0004429|1967|Dharmendra|  Male|
|tt0147811|nm0004429|1968|Dharmendra|  Male|
|tt0175908|nm0004429|1968|Dharmendra|  Male|
|tt0254437|nm0004429|1968|Dharmendra|  Male|
|tt0148841|nm0004429|1968|Dharmendra|  Male|
|tt0229950|nm0004429|1969|Dharmendra|  Male|
|tt0244776|nm0004429|1969|Dharmendra|  Male|
|tt0158449|nm0004429|1969|Dharmendra|  Male|
|tt0158375